# Machine Learning Project: Key Detection
Authors: Zach Hayes, Matt Gaetano, Max Ivry

Course: CS3540


In [2]:
import pandas as pd
import numpy as np

### Gather Training and Testing Data
Data cleaning and feature extraction occured earlier in process. See clean.ipynb

In [ ]:
def retrieve_data():
    train_df = pd.read_json("dataset/train.json", lines=True)
    test_df = pd.read_json("dataset/test.json", lines=True)

    X_train = train_df.loc[:, :-1]
    y_train = train_df.loc[:, -1:]

    X_test = test_df.loc[:, :-1]
    y_test = test_df.loc[:, -1:]

    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = retrieve_data()

In [11]:
train = pd.read_json("data.json", lines=True)

df[['note', 'modality']] = pd.DataFrame(df.target.tolist(), index=df.index)
print(df["target"].value_counts())
print(df["note"].value_counts())
print(df["modality"].value_counts())

C:\Users\Zachary\AppData\Local\Temp\ipykernel_16048\271156794.py:1: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df = pd.read_json("data.json", lines=True)
C:\Users\Zachary\AppData\Local\Temp\ipykernel_16048\271156794.py:1: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df = pd.read_json("data.json", lines=True)
C:\Users\Zachary\AppData\Local\Temp\ipykernel_16048\271156794.py:1: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future

target
[G, major]     763
[C, major]     676
[A, major]     670
[D, major]     664
[E, major]     602
[F, major]     548
[A, minor]     488
[B, major]     425
[C, minor]     366
[G, minor]     344
[D, minor]     337
[C#, major]    305
[F#, major]    304
[Bb, major]    276
[E, minor]     272
[D#, major]    257
[A#, major]    256
[B, minor]     249
[F, minor]     248
[G#, major]    226
[Ab, major]    191
[F#, minor]    191
[C#, minor]    188
[Eb, major]    178
[A#, minor]    155
[D#, minor]    135
[G#, minor]    134
[Ab, minor]     89
[Eb, minor]     76
[Bb, minor]     72
[Db, major]     66
[Gb, major]     50
[Gb, minor]     25
[Db, minor]     21
Name: count, dtype: int64
note
A     1158
G     1107
C     1042
D     1001
E      874
F      796
B      674
F#     495
C#     493
A#     411
D#     392
G#     360
Bb     348
Ab     280
Eb     254
Db      87
Gb      75
Name: count, dtype: int64
modality
major    6457
minor    3390
Name: count, dtype: int64


### Support Vector Machine Approach